In [ ]:
''' demo notebook to test adam path setup configuration '''

In [1]:
import sys
from os.path import expanduser
adam_home_defined = expanduser("~") + "/adam_home" # / syntax works for Windows, Mac, and Linux
sys.path.append(adam_home_defined) 
#import adam modules after setting adam directory path 
from adam import adam_config
# Set up adam paths
ADAMpaths = adam_config.setPaths.initPaths(adam_home_defined)

Changing adam home path to =  C:\Users\macuser/adam_home


In [2]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
import time
import os
import numpy as np
from datetime import datetime
import win32com.client as win32

In [3]:
#--------------------------------------------
# Path Directory from adam_config.json
#--------------------------------------------

#print('Path to ADAM package ' + adam_config.setPaths.home)
print('Path to ADAM package ' + ADAMpaths[0])

#print('Path to data directory ' + adam_config.setPaths.data_path)
print('Path to data directory ' + ADAMpaths[1])

#print('Path to environment and path configuration file ' + adam_config.setPaths.env_config_path)
print('Path to environment and path template file ' + ADAMpaths[2])

#--------------------------------------------
# User add file name for generated data files
user_folder_name = 'demo_ephem'
#--------------------------------------------

Path to ADAM package C:\Users\macuser/adam_home/
Path to data directory C:\Users\macuser/adam_home/data
Path to environment and path configuration file C:\Users\macuser/adam_home/config/adam_config_template.json


In [4]:
#--------------------------------------------
# THIS BLOCK WILL CREATE A NEW FOLDER EACH 
# TIME IT IS RUN - Daily resolution
#--------------------------------------------
# User add file name for generated data files
user_folder_name = 'demo_ephem'
# Add 8 character time stamp (YYYYMMDD) with data directory path
#data_folder = adam_config.setPaths.data_path + '/' + user_folder_name + '_'+ datetime.now().strftime("%Y_%m_%d")
data_folder = ADAMpaths[1] + '/' + user_folder_name + '_'+ datetime.now().strftime("%Y_%m_%d")
#data_folder = adam_config.setPaths.data_path + '/' + user_folder_name + '_'+ datetime.now().strftime("%Y_%m_%d_%H%M%s")

# Create data_folder with specified project name
if not os.path.isdir(data_folder):
    os.mkdir(data_folder)
    print("Directory created for TODAY's generated data: " + data_folder)
else:
    print("Directory " + data_folder + " already exists." )

Directory created for TODAY's generated data: C:\Users\macuser/adam_home/data/demo_ephem_2018_08_22


In [5]:
#Set up token and UUID from adam_config.json
#sys.path.append(adam_config.setPaths.env_config_path)
sys.path.append(ADAMpaths[3])
config = ConfigManager(ADAMpaths[3]).get_config()
auth_rest = AuthenticatingRestProxy(RestRequests(config.get_url()), config.get_token())#Open STK on the Desktop

In [6]:
# Start STK on the desktop (on Microsoft Windows)
stk_app = win32.gencache.EnsureDispatch('STK11.Application')
stk_app.Visible = True
print(stk_app.__module__)
STK = stk_app.Personality
root = stk_app.Personality2

win32com.gen_py.781C4C18-C2C9-4E16-B620-7B22BC8DE954x0x1x0._IAgUiApplication


In [7]:
# Load scenerio into STK 
scenarioPath = "/ADAM_HelioGraphics/ADAM_HelioGraphics.sc"
#fullScenarioPath = "{}{}".format(adam_config.setPaths.data_path,scenarioPath)
fullScenarioPath = "{}{}".format(ADAMpaths[1],scenarioPath)
root.LoadScenario(fullScenarioPath)
scenario = root.CurrentScenario

In [8]:
# Get the STK satellite object
g_gatorDefn = STK.scenario(0).Satellite('KeplerianConvert').GetAstrogatorDefn()
seg = g_gatorDefn.GetSegmentByName("Initial State")

In [ ]:
propagation_params = PropagationParams({
    'start_time': '2018-01-01T00:00:00Z',   # propagation start time in ISO format
    'end_time': '2020-01-01T00:00:00Z',     # propagation end time in ISO format

    'project_uuid': config.get_workspace(),

    'step_size': 86400,                   # step size (seconds)
    
#     'propagator_uuid': '00000000-0000-0000-0000-000000000002',  # force model
    
     'description': 'Visulaization of synthetic asteroids'       # description of run
})

In [ ]:
# Asteroid Population Generation from Sarah Greenstreet:

num_asteroids_to_generate = 1000

# Set up lists for the new orbital elements
sma3 = [] # semimajor axis (AU)
ecc3 = [] # eccentricity
inc_sun3 = [] # inclination (deg)
Omega3 = [] # longitude of the ascending node (deg)
arg_peri3 = [] # argument of perihelion (deg)
ta3 = [] # true anamoly (deg)

# Fill orbital element lists with 1000 random numbers drawn from a gaussian distribution
# Orbital element variables:
# sma3: mu=2.25, sigma=0.55
# ecc3: mu=0.45, sigma=0.15
# inc3: mu=10.0, sigma=3.0
# 0.0 <= Omega3 <= 359.0
# 0.0 <= arg_peri3 <= 359.0
# 0.0 <= ta3 <= 359.0
i = 0
while i < num_asteroids_to_generate:
    sma3.append(np.random.normal(2.25, 0.55))
    ecc3.append(np.random.normal(0.45, 0.15))
    inc_sun3.append(np.random.normal(10.0, 3.0))
    Omega3.append(np.random.uniform(low=0.0,high=359.0))
    arg_peri3.append(np.random.uniform(low=0.0,high=359.0))
    ta3.append(np.random.uniform(low=0.0,high=359.0))
    i = i + 1
    
# Now cut this sample down to only include NEOs (pericenter < 1.3 AU) that fall within a specified range of orbital element values.
sma4 = [] # semimajor axis (AU)
ecc4 = [] # eccentricity
inc_sun4 = [] # inclination (deg)
Omega4 = [] # longitude of the ascending node (deg)
arg_peri4 = [] # argument of perihelion (deg)
ta4 = [] # true anamoly (deg)

# Only keep those asteroids which are NEOs (pericenter < 1.3 AU) with 0.8 <= sma <= 2.1, 0.01 <= ecc <= 0.5, inc <= 20.0
j = 0
while j < num_asteroids_to_generate:
    pericenter = sma3[j] * (1.0-ecc3[j])
    if pericenter < 1.3: # (AU)
        if sma3[j] <= 2.1 and sma3[j] >= 0.8:
            if ecc3[j] <= 0.5 and ecc3[j] >= 0.01:
                if inc_sun3[j] <=20.0:
                    sma4.append(sma3[j])
                    ecc4.append(ecc3[j])
                    inc_sun4.append(inc_sun3[j])
                    Omega4.append(Omega3[j])
                    arg_peri4.append(arg_peri3[j])
                    ta4.append(ta3[j])
    j = j + 1

print('sma4[0]: {}, ecc4[0]: {}, inc_sun4[0]: {}, Omega4[0]: {}, arg_peri4[0]: {}, ta4[0]: {}'.format(sma4[0],ecc4[0],inc_sun4[0],Omega4[0],arg_peri4[0],ta4[0]))
print(len(sma4))    

In [ ]:
#####################################
# Enter number to submit to the cloud

num_to_send_to_cloud = 5

#####################################

if num_to_send_to_cloud > len(sma4):
    print("Not enough asteroids generated to send to cloud; Only {} generated but () requested".format(len(sma4),
                                                                                                num_to_send_to_cloud))
    num_to_send_to_cloud = len(sma4)

In [ ]:
# Convert the Keplerian Orbital Elements to Cartesain for ADAM, and create batch          
batch = []

for x in range(0, num_to_send_to_cloud):
    tf = seg.InitialState.Keplerian.sma.Set(sma4[x], "Au")
    tf = seg.InitialState.Keplerian.ecc.value = ecc4[x]
    tf = seg.InitialState.Keplerian.inc.Set(inc_sun4[x], "deg")
    tf = seg.InitialState.Keplerian.RAAN.Set(Omega4[x], "deg")
    tf = seg.InitialState.Keplerian.w.Set(arg_peri4[x], "deg")
    tf = seg.InitialState.Keplerian.TA.Set(ta4[x], "deg")

    tf = g_gatorDefn.RunMCS()
    state_vec = [0.0,0.0,0.0,0.0,0.0,0.0]

    state_vec[0] = seg.InitialState.GetResultValue('Sun X').GetIn("km").value
    state_vec[1] = seg.InitialState.GetResultValue('Sun Y').GetIn("km").value
    state_vec[2] = seg.InitialState.GetResultValue('Sun Z').GetIn("km").value
    state_vec[3] = seg.InitialState.GetResultValue('Sun Vx').GetIn("km/sec").value
    state_vec[4] = seg.InitialState.GetResultValue('Sun Vy').GetIn("km/sec").value
    state_vec[5] = seg.InitialState.GetResultValue('Sun Vz').GetIn("km/sec").value

    opm_params = OpmParams({
        'epoch': '2018-01-01T00:00:00Z',
        'state_vector': state_vec,

    #     'mass': 500.5,              # object mass
    #     'solar_rad_area': 25.2,     # object solar radiation area (m^2)
    #     'solar_rad_coeff': 1.2,     # object solar radiation coefficient
    #     'drag_area': 33.3,          # object drag area (m^2)
    #     'drag_coeff': 2.5,          # object drag coefficient

    #     'covariance': covariance,   # object covariance
    #     'perturbation': 3,          # sigma perturbation on state vector
    #     'hypercube': 'FACES',       # hypercube propagation type

    #     'originator': 'Robot',      # originator of run
    #     'object_name': 'TestObj',   # object name
    #     'object_id': 'test1234',    # object ID
    })

    batch.append(Batch(propagation_params, opm_params))
    #batch = Batch(propagation_params, opm_params)
batch

In [ ]:
print("Submitting OPM:")
print(batch[0].get_opm_params().generate_opm())

In [ ]:
propagation_params

In [ ]:
# Submit runs to the cloud and wait until batch run is ready

# Start the timer so we know how long this took to run.
startTime = datetime.now()
print("Started run: {}".format(str(startTime)))

batches_module = Batches(auth_rest)
#BatchRunManager(batches_module, [batch]).run()
BatchRunManager(batches_module, batch).run()

# Display the time and elasped time
stopTime = datetime.now()
elapsedTime = (stopTime-startTime)
print("Current Time: {}; Elapsed Time: {} minutes".format(str(stopTime),elapsedTime.total_seconds()/60.0))

In [ ]:
# Get final status and parts count
for b in batch:
    parts_count = b.get_state_summary().get_parts_count()
    print("Final state: %s, part count %s\n" % (b.get_calc_state(), parts_count))

In [ ]:
# Get ephemeris of specified part
part_to_get = 0
eph = batch[0].get_results().get_parts()[part_to_get].get_ephemeris()
print("Ephemeris:")
print(eph)

In [ ]:
# Get the end state vector (uncomment to use)
for b in batch:
    end_state_vector = b.get_results().get_end_state_vector()
    print("State vector at the end of propagation:")
    print(end_state_vector)

In [ ]:
# Write the ephemeris to a STK .e file
countNum = 0 #ek 060618 added to get around NameError: not defined
countNum = countNum + 1 # Increment count for unique ephemeris and STK satellite object name

satName = '{:04d}'.format(countNum)
fileName = '{}.e'.format(satName)
#filePath = adam_config.setPaths.data_path

print('STK satellite object name: {}    Ephemeris File Name: {}   Ephemeris File Path: {}'.format(satName,fileName, data_folder))

with open(data_folder + '/' + fileName, 'w') as f:
        f.write(eph)

In [ ]:
# Use STK MTO (Multi Track Object) to store epehmeris points in STK

mto = root.CurrentScenario.Children('AsteroidTracks')
root.ExecuteCommand('VO */MTO/AsteroidTracks System "CentralBody/Sun Inertial"')

#newMTO = mto.CopyObject('Temp') # satName)

In [ ]:
# Save ephemeris to STK ".e" files, and add as track to MTO object in STK
countNum = 1 

for b in batch:
    #Wrtie ephemeris to file
    eph = b.get_results().get_parts()[0].get_ephemeris()
    satName = '{:04d}'.format(countNum)
    fileName = '{}.e'.format(satName)
    print('STK satellite object name: {}    Ephemeris File Name: {}'.format(satName,fileName))
    with open(data_folder + '/' + fileName, 'w') as f:
        f.write(eph)
    
    # Add ephemeris file as MTO track
    newTrack = mto.Tracks.Add(countNum)
    newTrack.Points.LoadPoints(data_folder + '/' + fileName)
    
    # Set graphics properties of track
    newTrack.Interpolate = True
    mto.Graphics.Tracks.GetTrackFromId(countNum).Line.Width = 1
    mto.Graphics.Tracks.GetTrackFromId(countNum).Color = 0xe16941 # Blue Green Red: 225 105 065
    mto.Graphics.Tracks.GetTrackFromId(countNum).Line.Translucency = 33
    
    # Increment count for unique ephemeris and STK satellite object name
    countNum = countNum + 1 

In [ ]:
# Quit STK after demonstration
stk_app.Visible = False
stk_app.Quit